In [0]:
import requests
import json
import time

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql.functions import col, when
from pyspark.sql.functions import concat


spark= SparkSession \
       .builder \
       .appName("PySpark Server") \
       .getOrCreate()

spark


In [0]:

next = True
page_count=100
payload = {"options":{"cellFormat":"string"},"pageContext":None,"pagingOption":{"offset":None,"count":page_count},"filterCriteria":{}} 
limit =5
count=0
data = []
while next and count<=limit:
    r=requests.post(url="""https://profiles.sunnahnikah.co/v1/datasource/airtable/53c28e05-67eb-4322-be23-ea1ac768c030/5a6b22a4-3c1c-411a-b1f0-2671ada07826/7f5c2698-daa0-408d-856d-f4a773818f77/eee4fb36-5b78-4c00-81d3-2f59d5b5ade6/data""", json=payload).json()
    for i in range(page_count):
        data.append((r.get("records")[i]["id"],r.get("records")[i].get("fields").get("Ethnicity"),r.get("records")[i].get("fields").get("Age"),r.get("records")[i].get("fields").get("Gender")))
    if r.get("offset") is None:
        next=False
    payload["pagingOption"]["offset"] = r["offset"]
    print(data)
    time.sleep(2)
    count+=1




In [0]:
print(data)

In [0]:
columns = ['id', 'Ethnicity', 'Age','Gender' ]
profiles_df = spark.createDataFrame(data = data, schema= columns)
profiles_df.show(1000)

In [0]:
profiles_df.write.mode("overwrite").format("csv").option("header", "true").save("/Volumes/sn_demo/default/vol1/sn_profiles.csv")